## Load Packages

In [2]:
using JuMP;
using HiGHS;

João is a farmer from a small town who specializes in growing wheat, corn and sugar beet. He owns $500 km^2$ of land and must decide the amount of land to be allocated to each of the crops. João faces several restrictions regarding his planting. First, he must have at least $200 T$ of wheat and $240 T$ of corn to feed his cattle. Such quantities can be obtained through own plantation or by buying from the city's cooperative. The purchase prices per ton of wheat are $238 R\$/T$ and per ton of corn $210 R\$/T$. On the other hand, any excess produced in relation to the minimum can be sold at the cooperative, however with a $40\%$ discount on the purchase price ($170 R\$/T$ for wheat and $150 R\$/T$ for corn) per the cooperative's margin account. Another important restriction concerns the sale of sugar beet. By law, the sale price of a ton of beet at the cooperative is fixed at $36 R\$/T$ for the first $6000 T$ sold. After this amount, the sale price becomes $10 R\$/T$. In addition, the cooperative where João trades his products does not have sugar beet for purchase.

Let's assume that the planting cost of each crop is: $150 R\$/km^2$ for wheat, $230 R\$/km^2$ for corn, and $260 R\$/km^2$ for sugar beet. The uncertainty of the problem lies in the productivity of the land. João does not know a priori how much each $km^2$ of land will yield in tons of culture. Now assume that three scenarios of equal probabilities of occurrence were sampled: "good", "average" and "poor". In each of the states, the yield of each crop is given by:
"Bad" state: Wheat = $2 T/km^2$; Corn = $2.4 T/km^2$; Beetroot = $16 T/km^2$;
"Average" state: Wheat = $2.5 T/km^2$; Corn = $3.0 T/km^2$; Beetroot = $20 T/km^2$;
"Good" state: Wheat = $3 T/km^2$; Corn = $3.6 T/km^2$; Beetroot = $24 T/km^2$;

## 1) Formulate the first and second stage problem. Additionally, considering the probability distribution defined above, formulate and solve the deterministic equivalent problem.
## Compute the value of the stochastic solution.
## Compute the expected value of perfect information.

$$
\begin{aligned}
    &\min && cx + \Sigma_s p_s(-qy_s-rz_s)  \\
    & st && 0 \leq x \leq u \\
        &&& y_s \leq d_s \\
        &&& y_s+z_s \leq x\\
        &&& y_s, z_s \geq 0\\
\end{aligned}
$$

In [24]:
struct NewsboyData
    u::Float64
    c::Float64
    q::Float64
    r::Float64
    d::Vector{Float64}
    p::Vector{Float64}
end

function get_newsboy_data(data::NewsboyData)
    u = data.u
    c = data.c
    q = data.q
    r = data.r
    d = data.d
    p = data.p
    return u, c, q, r, d, p
end

function newsboy_optimal(data::NewsboyData)
    
    u, c, q, r, d, p = get_newsboy_data(data)
    S = length(d)
    
    model = Model(HiGHS.Optimizer)
    MOI.set(model, JuMP.MOI.Silent(), true)
    @variable(model, 0<=x<=u)
    @variable(model, 0<=y[s=1:S]<=d[s])
    @variable(model, 0<=z[1:S])

    @constraint(model,[s = 1:S], y[s] + z[s] <= x)

    @objective(model, Min,c*x - sum(q*y[s]*p[s]+r*z[s]*p[s] for s = 1:S))

    optimize!(model)
    return objective_value(model), value(model[:x])
end

function newsboy_crystal_ball(data::NewsboyData)
    
    u, c, q, r, d, p = get_newsboy_data(data)
    S = length(d)
    
    model = Model(HiGHS.Optimizer)
    MOI.set(model, JuMP.MOI.Silent(), true)
    @variable(model, 0<=x[s=1:S]<=u)
    @variable(model, 0<=y[s=1:S]<=d[s])
    @variable(model, 0<=z[1:S])

    @constraint(model,[s = 1:S], y[s] + z[s] <= x[s])

    @objective(model, Min, sum(c*x[s]*p[s] - q*y[s]*p[s]-r*z[s]*p[s] for s = 1:S))

    optimize!(model)
    return objective_value(model)
end

function newsboy_average(data::NewsboyData)
    
    u, c, q, r, d, p = get_newsboy_data(data)
    S = length(d)
    x = min(u,sum(p.*d))
      
    model = Model(HiGHS.Optimizer)
    MOI.set(model, JuMP.MOI.Silent(), true)
    @variable(model, 0<=y[s=1:S]<=d[s])
    @variable(model, 0<=z[1:S])

    @constraint(model,[s = 1:S], y[s] + z[s] <= x)

    @objective(model, Min, c*x - sum(q*y[s]*p[s]+r*z[s]*p[s] for s = 1:S))

    optimize!(model)
    return objective_value(model)
end

newsboy_average (generic function with 1 method)

In [30]:
data = NewsboyData(150,10,25,5,LinRange(60,150,91),ones(91)./91)
obj1, x = newsboy_optimal(data)
obj2 = newsboy_crystal_ball(data)
obj3 = newsboy_average(data)
println("x = $(x)")
println("Optimal Solution = $(obj1)")
println("Wait and See Solution = $(obj2)")
println("Average demand Solution = $(obj3)")
println("VSS = $(obj1-obj2)")
println("EPVI = $(obj3-obj1)")

x = 128.0
Optimal Solution = -1404.3956043956048
Wait and See Solution = -1574.9999999999998
Average demand Solution = -1347.5274725274721
VSS = 170.604395604395
EPVI = 56.86813186813265
